# Actividad 2 — Árbol de Decisión  
**Autor:** Fabricio Bermúdez

Este notebook entrena y evalúa un **Árbol de Decisión** para la detección de anomalías en logs de HDFS, usando SIEMPRE el mismo dataset de la actividad:

- `dataset/preprocessed/Event_occurrence_matrix.csv`
- `dataset/preprocessed/anomaly_label.csv`

> *Nota:* Si ejecutas este notebook aquí mismo, también intentará una ruta alternativa (`/mnt/data/dataset_extracted/dataset/preprocessed`) para encontrar los archivos que ya usé en esta sesión.


In [ ]:
# ========================
# 0) Importaciones y setup
# ========================
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

pd.set_option('display.max_columns', 200)


In [ ]:
# ==========================================
# 1) Carga de datos (dos rutas posibles)
# ==========================================
rel_base = os.path.join('dataset', 'preprocessed')
alt_base = '/mnt/data/dataset_extracted/dataset/preprocessed'

def try_read(base_path):
    occ = pd.read_csv(os.path.join(base_path, 'Event_occurrence_matrix.csv'))
    lab = pd.read_csv(os.path.join(base_path, 'anomaly_label.csv'))
    return occ, lab

base_used = None
try:
    df_occ, df_lab = try_read(rel_base)
    base_used = rel_base
except Exception as e1:
    try:
        df_occ, df_lab = try_read(alt_base)
        base_used = alt_base
    except Exception as e2:
        raise FileNotFoundError(
            f'No se pudieron leer los CSV. Verifica que existan en {rel_base} o {alt_base}.\n'
            f'Errores:\n- {e1}\n- {e2}'
        )

print(f'✅ Datos cargados desde: {base_used}')
print('Shapes -> occ:', df_occ.shape, 'labels:', df_lab.shape)
display(df_occ.head(3))
display(df_lab.head(3))


In [ ]:
# ==========================================
# 2) Preparación de datos
# ==========================================
df = df_occ.merge(df_lab, on='BlockId', suffixes=('', '_true'))

cols_excluir = ['BlockId', 'Label', 'Label_true', 'Type']
feature_cols = [c for c in df.columns if c not in cols_excluir]
X = df[feature_cols]
y = df['Label_true'] if 'Label_true' in df.columns else df['Label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

X.shape, X_train.shape, X_test.shape


In [ ]:
# ==========================================
# 3) Entrenamiento del Árbol de Decisión
# ==========================================
tree_clf = DecisionTreeClassifier(
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)
tree_clf.fit(X_train, y_train)

y_pred = tree_clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print('=== Métricas del Árbol de Decisión ===')
print(f'Accuracy: {acc:.6f}')
print('\nClassification report:')
print(report)

print('\nMatriz de confusión:')
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=tree_clf.classes_)
disp.plot(values_format='d')
plt.title('Matriz de confusión — Árbol de Decisión')
plt.show()


In [ ]:
# ==========================================
# 4) Importancia de variables
# ==========================================
importances = pd.Series(tree_clf.feature_importances_, index=feature_cols).sort_values(ascending=False)
top10 = importances.head(10)
print('Top 10 features más importantes:')
display(top10.to_frame('importance'))

top10.plot(kind='bar')
plt.title('Top 10 Importancias — Árbol de Decisión')
plt.ylabel('Importancia')
plt.xlabel('Feature')
plt.tight_layout()
plt.show()


In [ ]:
# ==========================================
# 5) Árbol reducido (reglas interpretables)
# ==========================================
small_tree = DecisionTreeClassifier(
    criterion='gini',
    max_depth=4,
    random_state=42
)
small_tree.fit(X_train, y_train)

rules = export_text(small_tree, feature_names=list(feature_cols))
print('=== Reglas del árbol (max_depth=4) ===')
print(rules)

plt.figure(figsize=(14, 8))
plot_tree(small_tree, feature_names=feature_cols, class_names=small_tree.classes_, filled=False, impurity=True)
plt.title('Árbol de Decisión (max_depth=4)')
plt.show()


## 6) Conclusiones

- El Árbol de Decisión alcanza una **alta precisión** sobre el dataset de HDFS.
- La interpretación es directa a través de **reglas** y **importancias**.
- Para el informe, se recomienda:
  - Relacionar las `E*` más importantes con sus plantillas en `HDFS.log_templates.csv`.
  - Reportar **Accuracy, F1, matriz de confusión** y ejemplos de reglas.
  - Describir el procedimiento para prevenir **fugas de información** (no usar `BlockId`, `Type` como features).

**Autor:** Fabricio Bermúdez
